# Rigorous estimates of p-laplacian's first eigenvalue

In [ ]:
using DifferentialEquations
using Plots; gr()
using IntervalArithmetic
using LinearAlgebra
using LaTeXStrings
using Interact

**poznámky - plaplace_solve**
- numerické řešení na jakémkoliv intervalu (měl by být minimálně [0,1]) 
- výsledek je restrikce řešení na [0,1]

In [ ]:
"""
Numeric solution of p-laplace

Inputs:
λi  ... initial interval of λ₁
p   ... p of p-laplacian
n   ... number of solution points
u₂0 ... initial condition for u₂
dom ... domain

Outputs:
t, tᴵ    ... solution points (Float64, Interval)
U₁, U₁ᴵ  ... numerical u₁ solution at t, tᴵ 
U₂, U₂ᴵ  ... numerical u₂ solution at t, tᴵ
Λ₁       ... numerical first eigenvalue λ₁
"""
function plaplace_solve(λi, p, n; u₂0=1.0, dom=(0.0, 1.0))
   
    function sl(du,u,P,t) # parametr p je odhad na λ₁ a p z p-laplacianu
        λ, p = P
        du[1] = abs(u[2])^(1/(p-1)) * sign(u[2])
        du[2] = -λ * abs(u[1])^(p-1)*sign(u[1]) 
    end
    
    tl, tr = dom
    
    u0 = [0.0; u₂0;] # počáteční podmínka
    a, b = λi
    Λ₁ = (a + b)/2
    Δt = (tr-tl)/(n-1) # velikost intervalu dělení
    e = 1e-12 # zastavovací podmínka

    while (b-a) >= e
        prob = ODEProblem(sl, u0, dom, (Λ₁, p))
        sol = solve(prob, saveat=Δt, abstol=1e-10,reltol=1e-10)
        if sol(tr)[1] == 0
            break
        else
            probA = ODEProblem(sl, u0, dom, (a, p))
            solA = solve(probA, saveat=Δt, abstol=1e-8,reltol=1e-8)
            probS = ODEProblem(sl, u0, dom, (Λ₁, p))
            solS = solve(probS, saveat=Δt, abstol=1e-8,reltol=1e-8)
            if solA(tr)[1] * solS(tr)[1] < 0
                b = Λ₁
            else
                a = Λ₁
            end
            Λ₁ = (a+b)/2
        end
    end

    prob = ODEProblem(sl, u0, dom, (Λ₁, p))
    sol = solve(prob, saveat=Δt, abstol=1e-8,reltol=1e-8)
    
    t = LinRange(0,1,n-1)
    tᴵ = [@interval(i) for i in t]
    U₁ = [u[1] for u in sol(t).u]
    U₁ᴵ = [@interval(u[1]) for u in sol(t).u]
    U₂ = [u[2] for u in sol(t).u]
    U₂ᴵ = [@interval(u[2]) for u in sol(t).u]

    
    return t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, sol
end;

In [ ]:
"""
Intervalově zadané řídící body proloží intervalově kubickým splinem.

Vstupní parametry:
t, tᴵ         ... bodové a intervalové dělení časové osy
U, Uᴵ         ... body a intervaly které prokláme splinem
Uₗd2, Uᵣd2    ... hodnoty druhé derivace v krajních (levý, pravý) bodech
ns=10         ... dělení jednotlivých oblouků kubického splinu

Výstupní hodnoty:
csc_u ... koeficienty kubického splinu
Uˢ   ... intervalové hodnoty splinu v bodech dělení
"""
function cubic_natural_spline(t, tᴵ, U, Uᴵ, Uₗd2, Uᵣd2; ns=10)
    #matice A
    n=length(Uᴵ)
    dv = [4..4 for i in 1:n-2]
    ev = [1..1 for i in 1:n-3]
    A = Array(SymTridiagonal(dv,ev))
    A⁻¹ = inv(A)

    #pravá strana
    h = 1.0/(n-1)
    rhs = []
    for i in 3:length(Uᴵ)
        append!(rhs, 6/h^2 * (Uᴵ[i] - 2 * Uᴵ[i-1] + Uᴵ[i-2]))
    end
    
    rhs[1] = rhs[1]-Uₗd2
    rhs[end] = rhs[end]-Uᵣd2    

    # vektor druhých derivací  
    Ud2 = []
    append!(Ud2, @interval(Uₗd2))
    append!(Ud2, A⁻¹*rhs)
    append!(Ud2, @interval(Uᵣd2))

    # koeficienty jednotlivých oblouků splinu
    csc_U = []
    for i in 1:length(Uᴵ)-1
        a=b=c=d=0
        a = (Ud2[i+1]-Ud2[i])/(6*h)
        b = Ud2[i]/2 
        c = (Uᴵ[i+1] - Uᴵ[i])/h - h*(2*Ud2[i]+Ud2[i+1])/6
        d = Uᴵ[i]
        append!(csc_U, [[a,b,c,d]])
    end 
    
    Uˢ = Interval[] # intervalově vyjádřený spline 
    for i in 1:length(Uᴵ)-1
        x_dom = t[i]..t[i+1]
        x_int = mince(x_dom,ns)
        f(x) = csc_U[i][4] + (x-t[i])*(csc_U[i][3] + (x-t[i])*(csc_U[i][2] + csc_U[i][1]*(x-t[i])))
        append!(Uˢ, f.(x_int))
    end
        
    return csc_U, Uˢ
end;

In [ ]:
"""
Intervalově zadané řídící body proloží intervalově kubickým splinem.

Vstupní parametry:
t, tᴵ         ... bodové a intervalové dělení časové osy
U, Uᴵ         ... body a intervaly které prokláme splinem
Uₗd1, Uᵣd1     ... hodnoty první derivace v krajních (levý, pravý) bodech
plt_res=true  ... vykreslení řešení
ns=10         ... dělení jednotlivých oblouků kubického splinu

Výstupní hodnoty:
csc_u ... koeficienty kubického splinu
Uˢ   ... intervalové hodnoty splinu v bodech dělení
"""
function cubic_end_slope_spline(t, tᴵ, U, Uᴵ, Uₗd1, Uᵣd1; ns=10)
    #matice A
    n=length(Uᴵ)
    dv = [4..4 for i in 1:n-2]
    ev = [1..1 for i in 1:n-3]
    A = Array(SymTridiagonal(dv,ev))
    A[1,1] = 3.5..3.5
    A[end,end] = 3.5..3.5
    A⁻¹ = inv(A)

    #pravá strana
    h = 1.0/(n-1)
    rhs = []
    for i in 3:length(Uᴵ)
        append!(rhs, 6/h^2 * (Uᴵ[i] - 2 * Uᴵ[i-1] + Uᴵ[i-2]))
    end
    
    rhs[1] = rhs[1] - 3/h * ( (Uᴵ[2]-Uᴵ[1])/h - Uₗd1)
    rhs[end] = rhs[end] - 3/h * (Uᵣd1 - (Uᴵ[end]-Uᴵ[end-1])/h)    

    # vektor druhých derivací  
    sol = A⁻¹*rhs
    Ud2 = []
    
    σ₀ = 3/h * ( (Uᴵ[2]-Uᴵ[1])/h - Uₗd1) - sol[1]/2
    σ₁ = 3/h * (Uᵣd1 - (Uᴵ[end]-Uᴵ[end-1])/h)   - sol[end]/2
    append!(Ud2, @interval(σ₀))
    append!(Ud2, sol)
    append!(Ud2, @interval(σ₁))

    # koeficienty jednotlivých oblouků splinu
    csc_U = []
    for i in 1:length(Uᴵ)-1
        a=b=c=d=0
        a = (Ud2[i+1]-Ud2[i])/(6*h)
        b = Ud2[i]/2 
        c = (Uᴵ[i+1] - Uᴵ[i])/h - h*(2*Ud2[i]+Ud2[i+1])/6
        d = Uᴵ[i]
        append!(csc_U, [[a,b,c,d]])
    end 
    
    Uˢ = Interval[] # intervalově vyjádřený spline 
    for i in 1:length(Uᴵ)-1
        x_dom = t[i]..t[i+1]
        x_int = mince(x_dom,ns)
        f(x) = csc_U[i][4] + (x-t[i])*(csc_U[i][3] + (x-t[i])*(csc_U[i][2] + csc_U[i][1]*(x-t[i])))
        append!(Uˢ, f.(x_int))
    end
        
    return csc_U, Uˢ
end;

**poznámky - get_u1**
- po nasčítání intervalového integrálu je třeba funkci "zespojitit" - cyklus se sjednocením dvou po sobě jdoucích intervalů
- s integrálem začínáme na 0 (u1 je posazené na osu)
- na konci odečítáme nejmenší dolní hodnotu U₁ˢ - ta je ovšem záporná, posun je tedy směrem nahoru a u1 >= 0

In [ ]:
"""
Rebuilds interval expression of U₁ˢ by integrating U₂ˢ.

Inputs:
p             ... p of p-laplacian
U₂ˢ           ... intervals of u₂ spline
t             ... time points of numeric solution
U₁            ... u₁(t) numeric values  

Output:
U₁ˢ  ... interval values of u₁(t)
"""
function get_u1(p, U₂ˢ, t, U₁)
    
    f(x) = abs(x)^(1/(p-1))*sign(x)
    ni = mince(0..1,length(U₂ˢ))

    U₁_tmp = Interval[0..0]#[U₁[1]..U₁[1]]
    for i in 1:length(U₂ˢ)
        append!(U₁_tmp, U₁_tmp[end] + f(U₂ˢ[i]) * diam(ni[i]))
    end
    
    U₁ˢ = Interval[]
    for i in 1:length(U₁_tmp)-1
        append!(U₁ˢ, U₁_tmp[i] ∪ U₁_tmp[i+1])
    end
    
    U₁ˢ = U₁ˢ .- inf(minimum(U₁ˢ))
    
    return U₁ˢ
end;

In [ ]:
"""
Z koeficientů intervalově vypočítá derivaci kubického splinu

Vstupní parametry:
csc           ... koeficienty splinu
t, tᴵ         ... bodové i intervalově dělení časové osy
ns            ... počet bodů dělení jednotlivého oblouku splinu
plt_res=true  ... vykreslení řešení


Výstupní hodnoty:
Uˢ   ... intervalové hodnoty derivace splinu

"""
function der_cubic_spline(csc, t, tᴵ, ns)
    
    Uₜₘₚ = Interval[]
    csc_Uder = [ [@interval(3) * c[1], @interval(2) * c[2], c[3]] for c in csc ] 
    for i in 1:length(t)-1
        x_dom = t[i]..t[i+1] 
        x_int = mince(x_dom,ns)
        f(x) = csc_Uder[i][3] + (x-t[i])*(csc_Uder[i][2] + (x-t[i])*csc_Uder[i][1])
        append!(Uₜₘₚ, f.(x_int))
    end
    
    Uˢ = Interval[]
    for i in 1:length(Uₜₘₚ)-1
        append!(Uˢ, Uₜₘₚ[i] ∪ Uₜₘₚ[i+1])
    end
    append!(Uˢ,Uₜₘₚ[end])
        
    return Uˢ
end;

In [ ]:
"""
Intervalově vypočítá dolní odhad λ₁

Vstupní parametry:
numerator     ... čitatel 
denominator   ... jmenovatel
plt_res=true  ... vykreslení řešení


Výstupní hodnoty:
λ₁ˡᵒʷ    ... dolní odhad λ₁
Fˡᵒʷ     ... intervalové hodnoty funkce dolního odhadu
"""
function lower_estimate(numerator, denominator, p)
    f(x,y) = -x / y^(p-1)
    λ₁ᵗᵐᵖ = f.(numerator, denominator)

    Fˡᵒʷ = Interval[]
    for i in 1:length(λ₁ᵗᵐᵖ)-1
        append!(Fˡᵒʷ, λ₁ᵗᵐᵖ[i] ∪ λ₁ᵗᵐᵖ[i+1])
    end
    append!(Fˡᵒʷ,λ₁ᵗᵐᵖ[end])
    
    λ₁ˡᵒʷ = inf(minimum(Fˡᵒʷ))
    
    return λ₁ˡᵒʷ, Fˡᵒʷ
end;

In [ ]:
"""
Intervalově vypočítá horní odhad λ₁

Vstupní parametry:
p             ... hodnota p z p-laplaciánu
U₁ˢ           ... intervalové hodnoty U₁
U₁ˢ_der       ... intervalové hodnoty U₁' 
plt_res=true  ... vypíše řešení


Výstupní hodnoty:
λ₁ᵘᵖ    ... horní odhad λ₁
"""
function upper_estimate(p, U₁ˢ, U₁ˢ_der)
    
    f(x) = abs(x)^(p)
    ni = mince(0..1,length(U₁ˢ))

    numerator = 0..0
    for i in 1:length(U₁ˢ_der)
        numerator = numerator + f(U₁ˢ_der[i]) * diam(ni[i])
    end
    
    denominator = 0..0
    for i in 1:length(U₁ˢ)
        denominator = denominator + f(U₁ˢ[i]) * diam(ni[i])
    end

    λ₁ᵘᵖ = sup(numerator/denominator)

    return λ₁ᵘᵖ 
end;

## Experiments

In [ ]:
l1low_f(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

l1s_low = Float64[]
l1s_low_err = Float64[]

l1s_up = Float64[]
l1s_up_err = Float64[]

n = 101 # bodů dělení 
ns = 100 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu

ps = 1.2:0.1:2.9
for p in ps
    λ₁ = l1low_f(p)
    λᵢₙᵢₜ = (3.,1.5*λ₁)
    r = 10/(n*ns)
    dom = (-r, r+1)

    ### lower estimate
    t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
    ud1 = -Λ₁ * U₁[end]^(p-1)
    csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);
    U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
    U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
    λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
    append!(l1s_low, λ₁ˡᵒʷ)
    append!(l1s_low_err, λ₁ˡᵒʷ-λ₁)

    ### upper estimate
    t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n);
    csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
    U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
    λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
    append!(l1s_up, λ₁ᵘᵖ)
    append!(l1s_up_err, λ₁ᵘᵖ-λ₁)

end


In [ ]:
plot(ps, l1low_f.(ps), label="přesné λ₁ₚ")
plot!(ps,l1s_up, line = :scatter, msw = 0, ms = 3, label="horní odhad λ₁ₚ")
plot!(ps, l1s_low, line = :scatter, msw = 0, ms = 3, 
        label="dolní odhad λ₁ₚ", fillrange = l1s_up, fillalpha = 0.2, 
        c = 3, legend=:bottomright, xlabel=L"p", ylabel=L"\lambda_1")

In [ ]:
plot(ps, l1low_f.(ps)-l1low_f.(ps), label="přesné λ₁ₚ")
plot!(ps,l1s_up_err, line = :scatter, msw = 0, ms = 3, label="chyba horního odhadu λ₁ₚ")
plot!(ps, l1s_low_err, line = :scatter, msw = 0, ms = 3, 
        label="chyba dolního odhadu λ₁ₚ", fillrange = l1s_up_err, fillalpha = 0.2, 
        c = 3, legend=:bottom, xlabel=L"p", ylabel=L"\lambda_1")

In [ ]:
plot(ps, l1low_f.(ps)-l1low_f.(ps), label="přesné λ₁ₚ")
plot!(ps,l1s_up_err, line = :scatter, msw = 0, ms = 3, label="horní odhad λ₁ₚ")
plot!(ps, l1s_low_err, line = :scatter, msw = 0, ms = 3, 
        label="dolní odhad λ₁ₚ", fillrange = l1s_up_err, fillalpha = 0.2, 
        c = 3, legend=:bottomleft, xlabel=L"p", ylabel=L"\lambda_1")

**Testy optimálních hodnot p=1.5**

pro n=22, ns=100
- minimální chyba = 0.024671607349520563
- optimální r = 0.005
- optimální ud1 na krajích = -0.4276

pro n=59, ns=100:
- minimální chyba = 0.010576760149503706
- optimální r = 0.0025118864315095794
- optimální ud1 na krajích = -0.31

In [ ]:
1/(59*100)

In [ ]:
@time begin
    p=1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p
    n = 22 # bodů dělení 
    ns = 100 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    results = []
    rs = 10. .^ -(1:0.2:5) #[0.005] 
    ud1s = -0.8:0.01:-0.1#03 #-0.43:0.0001:-0.42 #

    for r in rs

        res = Float64[]
        for ud1 in ud1s    

            dom = (-r, r+1)

            ### lower estimate
            t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); 

            csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);

            U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
            U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
            λ₁ˡᵒʷ, _ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
            append!(res, λ₁ˡᵒʷ)
        end

        append!(results, [res])
    end    
end

In [ ]:
# test pro n=59

@time begin
    p=1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p
    n = 59 # bodů dělení 
    ns = 100 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    results = []
    rs = 10. .^ -(1:0.2:5) #[0.005] 
    ud1s = -0.8:0.01:-0.1#03 #-0.43:0.0001:-0.42 #

    for r in rs

        res = Float64[]
        for ud1 in ud1s    

            dom = (-r, r+1)

            ### lower estimate
            t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); 

            csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);

            U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
            U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
            λ₁ˡᵒʷ, _ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
            append!(res, λ₁ˡᵒʷ)
        end

        append!(results, [res])
    end    
end

In [ ]:
@time begin
    p=1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p
    λᵢₙᵢₜ = (3.,1.5*λ₁); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)
    
    results = []
    #rs = 101. .^ -(1:0.2:5) #[0.005] 
    #ud1s = -0.8:0.01:-0.1#03 #-0.43:0.0001:-0.42 #

    NS = 400:100:1000
    for n in NS
        ns = 100 # bodů dělení jednotlivého oblouku spline
        si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
        
        r = 10/(n*ns)
        dom = (-r, r+1)

        ### lower estimate
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); 
        ud1 = -Λ₁ * U₁[end]^(p-1)
        csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);

        U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
        U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
        λ₁ˡᵒʷ, _ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
        append!(results, λ₁ˡᵒʷ)
    end
end

In [ ]:
plot(NS, λ₁.-results, title="p=$p, nejmenší chyba: $(minimum(λ₁.-results))", xlabel="n", ylabel="λ₁-λ₁ˡᵒʷ", legend=false)

In [ ]:
λ₁.-results

In [ ]:
errs = Float64[]
for i in 1:length(rs)
    append!(errs, minimum(λ₁ .- results[i]))
end

@show minimum(errs) #minimlní chyba
@show rs[argmin(errs)] #optimální r
@show ud1s[argmax(results[argmin(errs)])]; # optimální ud1 na krajích

In [ ]:
plot(rs, errs, label="error of estimate", xlabel="r", ylabel="error", xaxis=:log)

In [ ]:
ders = Float64[]
for i in 1:length(rs)
    append!(ders, ud1s[argmin(λ₁ .- results[i])])
end
plot(rs, ders, label="optimal ud1", xlabel="r", ylabel="ud1", xaxis=:log)

**Jde nastavit ud1 vzorečkem?**

U p=1.5 to zdá se vcelku funguje. Je třeba ale zvolit i správné r.

In [ ]:
#minimální chyba = 0.024671607349520563
#optimální r = 0.005
#optimální ud1 na krajích = -0.4276

p=1.5
λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

n = 359 # bodů dělení 
ns = 100 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

r = 0.0003
dom = (-r, r+1)

### lower estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, sol = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
Uₗd1 = Uᵣd1 = -Λ₁*U₁[end]^0.5# -0.299

csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
@show λ₁ˡᵒʷ
@show λ₁-λ₁ˡᵒʷ

### upper estimate
#t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
#csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
#U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
#λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
#@show λ₁ᵘᵖ
#@show λ₁ᵘᵖ-λ₁;

In [ ]:
1/(ns*n)

In [ ]:
-Λ₁*U₁[end]^0.5

### Testy optimálních hodnot p=3

- minimální chyba = 0.7173035185236429
- optimální r = 2.5e-11
- optimální ud1 na krajích = -0.00531

In [ ]:
@time begin
    p=3
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p
    n = 22 # bodů dělení 
    ns = 100 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,40.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    results = []
    rs = 10. .^ -(3:0.5:12) # [2.5e-11]
    ud1s = -0.05:0.001:-0.002 #-0.0055:0.00001:-0.0050

    for r in rs

        res = Float64[]
        for ud1 in ud1s    

            dom = (-r, r+1)

            ### lower estimate
            t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); 

            csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);

            U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
            U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
            λ₁ˡᵒʷ, _ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
            append!(res, λ₁ˡᵒʷ)
        end

        append!(results, [res])
    end    
end

In [ ]:
errs = Float64[]
for i in 1:length(rs)
    append!(errs, minimum(λ₁ .- results[i]))
end

@show minimum(errs) #minimlní chyba
@show rs[argmin(errs)] #optimální r
@show ud1s[argmax(results[argmin(errs)])]; # optimální ud1 na krajích
#0.7173078132029964

In [ ]:
plot(rs, errs, label="error of estimate", xlabel="r", ylabel="error", xaxis=:log)

In [ ]:
ders = Float64[]
for i in 1:length(rs)
    append!(ders, ud1s[argmin(λ₁ .- results[i])])
end
plot(rs, ders, label="optimal ud1", xlabel="r", ylabel="ud1", xaxis=:log)

In [ ]:
#minimální chyba = 0.7173035185236429
#optimální r = 2.5e-11
#optimální ud1 na krajích = -0.00531

p=3
λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

n = 22 # bodů dělení 
ns = 100 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
λᵢₙᵢₜ = (3.,40.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

r = 2.5e-11
dom = (-r, r+1)

### lower estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, sol = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
Uₗd1 = Uᵣd1 = -0.00531
csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
@show λ₁ˡᵒʷ
@show λ₁-λ₁ˡᵒʷ

### upper estimate
#t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
#csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
#U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
#λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
#@show λ₁ᵘᵖ
#@show λ₁ᵘᵖ-λ₁;

In [ ]:
ceil(Λ₁)*U₁[1]^(p-1)

In [ ]:
#minimální chyba = 0.7173035185236429
#optimální r = 2.5e-11
#optimální ud1 na krajích = -0.00531

p=3
λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

n = 22 # bodů dělení 
ns = 100 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
λᵢₙᵢₜ = (3.,40.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

r = 2.5e-11
dom = (-r, r+1)

### lower estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, sol = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
Uₗd1 = Uᵣd1 = -0.00531
csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
@show λ₁ˡᵒʷ
@show λ₁-λ₁ˡᵒʷ

### upper estimate
#t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
#csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
#U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
#λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
#@show λ₁ᵘᵖ
#@show λ₁ᵘᵖ-λ₁;

In [ ]:
#@show U₁ˢ[1]
#@show U₁ˢ[end]
aaa =  U₁ˢ .- inf(U₁ˢ[end])
bbb =  copy(U₁ˢ)
sup.(bbb)-sup.(aaa)

In [ ]:
@show inf(U₁ˢ[end]-inf(U₁ˢ[end])) == 0

In [ ]:
p=1.5
λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

n = 100 # bodů dělení 
ns = 1000 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

r = 0.003
dom = (-r, r+1)

### lower estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, sol = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
Uₗd1 = Uᵣd1 = -0.37641#-0.39#7641#1088
csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
@show λ₁ˡᵒʷ
@show λ₁-λ₁ˡᵒʷ

### upper estimate
#t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
#csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
#U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
#λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
#@show λ₁ᵘᵖ
#@show λ₁ᵘᵖ-λ₁;

In [ ]:
sol(0)

In [ ]:
Λ₁-λ₁ˡᵒʷ

In [ ]:
Λ₁-λ₁ˡᵒʷ

In [ ]:
oblouk = ceil(Int, argmin(inf.(Fˡᵒʷ))/ns)
@show oblouk
boxes = IntervalBox.(mince(0..1, length(Fˡᵒʷ)), Fˡᵒʷ)
plot(boxes, size=(1000,500), ylim=(5.27,5.32), xticks=(t,1:n-1), label="intervalově") #λ₁ˡᵒʷ/2,5*λ₁ˡᵒʷ/2  
cit = [-u[2] for u in sol.(LinRange(-r,1+r,n-1), Val{1})]
jme = [abs(u[1])^0.5 for u in sol.(LinRange(-r,1+r,n-1))]
Fnum = cit./jme

plot!(LinRange(0-r,1+r,n-1), Fnum, ylim=(2,8), label="numericky na větším intervalu")
ff(x)=Λ₁+0*x
plot!(LinRange(0-r,1+r,n), ff.(LinRange(0-r,1+r,n)), label="Λ₁", legend=:bottomright)

In [ ]:
Fnum[92]


In [ ]:
Λ₁

In [ ]:
[u[2] for u in sol.(LinRange(-r,1+r,1000), Val{1})][end-50:end]

In [ ]:
@manipulate for r=slider(0.001:0.001:0.1, value=0.005), ud1=slider(-0.95:0.01:-0.18, value=-0.39), ns = slider(10:10:100, value=100)

    p=1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

    n = 22 # bodů dělení 
    #ns = 10 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    #r = 0.005
    dom = (-r, r+1)

    ### lower estimate
    t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom);
    

    
    Uₗd1 = Uᵣd1 = ud1#-0.31088
    csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
    #tints = Interval[]
    #for i in 1:length(t)-1
    #    append!(tints, t[i]..t[i+1])
    #end
    
    
    U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
    U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
    λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)

    #boxes = IntervalBox.(mince(0..1, length(U₂ˢ)), U₂ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₁ˢ)), U₁ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₂ˢ_der)), U₂ˢ_der)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    boxes = IntervalBox.(mince(0..1, length(Fˡᵒʷ)), Fˡᵒʷ)
    plot(boxes, ylim=(3,12), xticks=(t,1:n) ,legend=false, size=(1000,500), 
        title = "λ₁ˡᵒʷ: $λ₁ˡᵒʷ , chyba: $(λ₁-λ₁ˡᵒʷ), oblouk: $(ceil(Int, argmin(inf.(Fˡᵒʷ))/ns)), ud1: $(Λ₁*U₁[end]^(p-1))")
    
end

In [ ]:
#@manipulate for r=0.001:0.001:0.1, ud1=-0.95:0.01:-0.18, ns = 10:10:100
@manipulate for r=0.001:0.001:0.1, ud1=-.3:0.001:-0.001, ns = 10:10:100
    p=3#1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

    n = 22 # bodů dělení 
    #ns = 10 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,40.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    #r = 0.005
    dom = (-r, r+1)

    ### lower estimate
    t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom);
    

    
    Uₗd1 = Uᵣd1 = ud1#-0.31088
    csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
    #tints = Interval[]
    #for i in 1:length(t)-1
    #    append!(tints, t[i]..t[i+1])
    #end
    
    
    U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
    U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
    λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)

    #boxes = IntervalBox.(mince(0..1, length(U₂ˢ)), U₂ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₁ˢ)), U₁ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₂ˢ_der)), U₂ˢ_der)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    boxes = IntervalBox.(mince(0..1, length(Fˡᵒʷ)), Fˡᵒʷ)
    #plot(boxes, ylim=(3,12), xticks=(t,1:n) ,legend=false, title = "λ₁: $λ₁ˡᵒʷ , chyba: $(λ₁-λ₁ˡᵒʷ)")
    plot(boxes, ylim=(20,35), xticks=(t,1:n) ,legend=false, size=(1000,500), 
        title = "λ₁ˡᵒʷ: $λ₁ˡᵒʷ , chyba: $(λ₁-λ₁ˡᵒʷ), oblouk: $(ceil(Int, argmin(inf.(Fˡᵒʷ))/ns)), ud1: $(Λ₁*U₁[1]^(p-1))")
    
end

In [ ]:
Λ₁*0.017^2

In [ ]:
Λ₁

In [ ]:
@manipulate for ud1=0.9:0.01:1.1, ns = 10:10:100

    p=1.5
    λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

    n = 22 # bodů dělení 
    #ns = 10 # bodů dělení jednotlivého oblouku spline
    si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
    λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

    #r = 0.005
    #dom = (-r, r+1)

    ### upper estimate
    t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁, _ = plaplace_solve(λᵢₙᵢₜ, p, n);
    #csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, ud2, ud2, ns=ns);
    csc_U₁, U₁ˢ = cubic_end_slope_spline(t, tᴵ, U₁, U₁ᴵ, ud1, -ud1, ns=ns);    
    U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
    λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
    chyba = λ₁ᵘᵖ-λ₁

    ##boxes = IntervalBox.(mince(0..1, length(U₁ˢ)), U₁ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₁ˢ)), U₁ˢ)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    #boxes = IntervalBox.(mince(0..1, length(U₁ˢ_der)), U₁ˢ_der)
    #plot(boxes, xticks=(t,1:n) ,legend=false)
    
    boxes = IntervalBox.(mince(0..1, length(Fˡᵒʷ)), Fˡᵒʷ)
    plot(boxes, ylim=(3,12), xticks=(t,1:n) ,legend=false, title = "λ₁: $λ₁ˡᵒʷ , chyba: $(λ₁-λ₁ˡᵒʷ)")
    
end

In [ ]:
p=1.5
λ₁ = (p-1)*(2*(π/p)/(sin(π/p)))^p

n = 22 # bodů dělení 
ns = 1000 # bodů dělení jednotlivého oblouku spline
si = mince(0..1,(n-1)*ns) # intervaly dělení splinu
λᵢₙᵢₜ = (3.,8.); # interval na kterém hledáme λ₁ (nesmí obsahovat vyšší λᵢ)

r = 0.003
dom = (-r, r+1)

### lower estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom);
Uₗd1 = Uᵣd1 =  -0.37641
csc_U₂ˢ, U₂ˢ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, Uₗd1, Uᵣd1, ns=ns);
U₁ˢ = get_u1(p,U₂ˢ, t, U₁);
U₂ˢ_der = der_cubic_spline(csc_U₂ˢ, t, tᴵ, ns);
λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(U₂ˢ_der, U₁ˢ, p)
@show λ₁ˡᵒʷ
@show λ₁-λ₁ˡᵒʷ

### upper estimate
t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
csc_U₁, U₁ˢ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, -1, -1, ns=ns);
U₁ˢ_der = der_cubic_spline(csc_U₁, t, tᴵ, ns); 
λ₁ᵘᵖ = upper_estimate(p, U₁ˢ, U₁ˢ_der);
@show λ₁ᵘᵖ
@show λ₁ᵘᵖ-λ₁;

In [ ]:
Λ₁ * 0.003^0.5

In [ ]:
oblouk = ceil(Int, argmin(inf.(Fˡᵒʷ))/ns)
@show oblouk
boxes = IntervalBox.(mince(0..1, length(Fˡᵒʷ)), Fˡᵒʷ)
plot(boxes, ylim=(λ₁ˡᵒʷ/2,5*λ₁ˡᵒʷ/2), xticks=(t,1:n) ,legend=false)

In [ ]:
using Interact
using Plots

In [ ]:
@manipulate for a=1:2:6
    f(x)=x^a
    plot(f) 
end

In [ ]:
using WebIO
WebIO.install_jupyter_labextension()


In [ ]:
import Pkg; Pkg.add("WebIO")

In [ ]:
using Pkg
Pkg.add("IJulia")